To run this notebook you first need to run the notebook `evaluation/compute_metrics.ipynb` or obtain the data generated for such notebook in the corresponding folders
---

In [34]:
from json import dump, load
from pprint import pprint
from statistics import median, mean, stdev
import networkx as nx

## Load gene graphs

f = open(f'../gene_graphs/vertices_inv.json', 'r')
vertices_inv = load(f)
f.close()
vertices_inv = {int(k): tuple(v) for k,v in vertices_inv.items()}

components = list()
for i in range(27121): ## Number of gene_graphs
    components.append(dict())
    components[i]['graph'] = nx.read_edgelist(f'../gene_graphs/graphs/component_{i+1}.edgelist', delimiter=':', create_using=nx.DiGraph, nodetype=int)
    components[i]['len'] = len(components[i]['graph'])
    
    f = open(f'../gene_graphs/sources/component_{i+1}.json', 'r')
    components[i]['sources'] = set(load(f))
    f.close()
    
    f = open(f'../gene_graphs/targets/component_{i+1}.json', 'r')
    components[i]['targets'] = set(load(f))
    f.close()
    
    f = open(f'../gene_graphs/vertex_constrains/component_{i+1}.json', 'r')
    components[i]['vertex_constrains'] = set(load(f))
    f.close()
    
    f = open(f'../gene_graphs/transcript_paths/component_{i+1}.json', 'r')
    components[i]['transcript_paths'] = load(f)
    f.close()
    

In [35]:
## Load experiments from json files
for i, component in enumerate(components):
    if component['len'] > 2:
        file = open(f'../safe_paths_json/component_{i+1}.json', "r")
        dd = load(file)
        component.update(dd)
        file.close()

In [36]:
def interval_length(interval):
    return interval[1]-interval[0]+1

## It computes the base length of a transcript
def base_length(contig, vertices_inv):
    length = 0
    for v in contig:
        length += interval_length(vertices_inv[v])
    return length

In [37]:
def choose_60_30_10_limits(components, vertices_inv):
    lengths = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            for transcript in component['transcript_paths']:
                lengths.append(base_length(transcript, vertices_inv))
    lengths = sorted(lengths)
    return lengths[int(0.6*len(lengths))], lengths[int(0.9*len(lengths))], lengths[-1]

limit_60, limit_30, limit_10 = choose_60_30_10_limits(components, vertices_inv)
limit_60, limit_30, limit_10

(1356, 3809, 205012)

In [38]:
def choose_60_30_10_limits_size(components, vertices_inv):
    sizes = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            sizes.append(component['len'])
    sizes = sorted(sizes)
    return sizes[int(0.6*len(sizes))], sizes[int(0.9*len(sizes))], sizes[-1]

limit_size_60, limit_size_30, limit_size_10 = choose_60_30_10_limits_size(components, vertices_inv)
limit_size_60, limit_size_30, limit_size_10

(21, 52, 725)

In [43]:
def compute_abs_improvement_table(over_width, limit_60, limit_30, components, vertices_inv):
    unitigs = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)

            safe_paths = component['experiments'][str(l)]['safe_paths']
            uni = component['unitigs']
            
            abs_vertex = component['experiments'][str(l)]['impr_vertex']
            abs_base = component['experiments'][str(l)]['impr_base']
            
            abs_vertex = list(map(lambda r:  r[1] - len(uni[r[0]]), enumerate(abs_vertex)))
            abs_base = list(map(lambda r:  r[1] - base_length(uni[r[0]], vertices_inv), enumerate(abs_base)))
            
                
            for j in range(len(abs_base)):
                d = dict()
                d['component'] = i
                d['component_length'] = component['len']
                d['abs_improvements_base'] = abs_base[j]
                d['abs_improvements_vertex'] = abs_vertex[j]
                
                unitigs.append(d)

    small = list(filter(lambda c: c['component_length'] <= limit_60, unitigs))
    medium = list(filter(lambda c: c['component_length'] > limit_60 and c['component_length'] <= limit_30 , unitigs))
    large = list(filter(lambda c: c['component_length'] > limit_30, unitigs))
    
    small_abs_improvements_base = list(map(lambda c: c['abs_improvements_base'] , small))
    medium_abs_improvements_base = list(map(lambda c: c['abs_improvements_base'] , medium))
    large_abs_improvements_base = list(map(lambda c: c['abs_improvements_base'] , large))
    total_abs_improvements_base = list(map(lambda c: c['abs_improvements_base'] , unitigs))
    
    small_abs_improvements_vertex = list(map(lambda c: c['abs_improvements_vertex'] , small))
    medium_abs_improvements_vertex = list(map(lambda c: c['abs_improvements_vertex'] , medium))
    large_abs_improvements_vertex = list(map(lambda c: c['abs_improvements_vertex'] , large))
    total_abs_improvements_vertex = list(map(lambda c: c['abs_improvements_vertex'] , unitigs))
    
    
    
    
    return { 
        'small':
        {
            'abs_improvements_vertex': 
            {
                'median': median(small_abs_improvements_vertex),
                'mean': mean(small_abs_improvements_vertex),
                'stdev': stdev(small_abs_improvements_vertex)
            },
            'abs_improvements_base':
            {
                'median': median(small_abs_improvements_base),
                'mean': mean(small_abs_improvements_base),
                'stdev': stdev(small_abs_improvements_base)
            },
            'number_of_unitigs': len(small)            
        },
        'medium':
        {
            'abs_improvements_vertex': 
            {
                'median': median(medium_abs_improvements_vertex),
                'mean': mean(medium_abs_improvements_vertex),
                'stdev': stdev(medium_abs_improvements_vertex)
            },
            'abs_improvements_base':
            {
                'median': median(medium_abs_improvements_base),
                'mean': mean(medium_abs_improvements_base),
                'stdev': stdev(medium_abs_improvements_base)
            },
            'number_of_unitigs': len(medium)
            
        },
        'large':
        {
            'abs_improvements_vertex': 
            {
                'median': median(large_abs_improvements_vertex),
                'mean': mean(large_abs_improvements_vertex),
                'stdev': stdev(large_abs_improvements_vertex)
            },
            'abs_improvements_base':
            {
                'median': median(large_abs_improvements_base),
                'mean': mean(large_abs_improvements_base),
                'stdev': stdev(large_abs_improvements_base)
            },
            'number_of_unitigs': len(large)
            
        },
        'total':
        {
            'abs_improvements_vertex': 
            {
                'median': median(total_abs_improvements_vertex),
                'mean': mean(total_abs_improvements_vertex),
                'stdev': stdev(total_abs_improvements_vertex)
            },
            'abs_improvements_base':
            {
                'median': median(total_abs_improvements_base),
                'mean': mean(total_abs_improvements_base),
                'stdev': stdev(total_abs_improvements_base)
            },
            'number_of_unitigs': len(unitigs)
            
        }
    }
    
pprint(compute_abs_improvement_table(0, limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'abs_improvements_base': {'mean': 620.3321457014713,
                                     'median': 214,
                                     'stdev': 1253.043242720963},
           'abs_improvements_vertex': {'mean': 5.018401307792262,
                                       'median': 3,
                                       'stdev': 7.826427851424918},
           'number_of_unitigs': 23857},
 'medium': {'abs_improvements_base': {'mean': 740.421323575017,
                                      'median': 225.0,
                                      'stdev': 1438.0371968407255},
            'abs_improvements_vertex': {'mean': 4.661694473733176,
                                        'median': 2.0,
                                        'stdev': 6.536794068539553},
            'number_of_unitigs': 32246},
 'small': {'abs_improvements_base': {'mean': 640.9470082749841,
                                     'median': 101.0,
                                     'stdev': 1357.9232

In [48]:
def compute_rel_improvement_table(over_width, limit_60, limit_30, components, vertices_inv):
    unitigs = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)

            uni = component['unitigs']
            safe_paths = component['experiments'][str(l)]['safe_paths']
            
            rel_vertex = component['experiments'][str(l)]['impr_vertex']
            rel_base = component['experiments'][str(l)]['impr_base']
            
            rel_vertex = list(map(lambda r:  r[1]/len(uni[r[0]]), enumerate(rel_vertex)))
            rel_base = list(map(lambda r:  r[1]/base_length(uni[r[0]], vertices_inv), enumerate(rel_base)))
            
            
                
            for j in range(len(rel_base)):
                d = dict()
                d['component'] = i
                d['component_length'] = component['len']
                d['rel_improvements_base'] = rel_base[j]
                d['rel_improvements_vertex'] = rel_vertex[j]
                
                unitigs.append(d)

    small = list(filter(lambda c: c['component_length'] <= limit_60, unitigs))
    medium = list(filter(lambda c: c['component_length'] > limit_60 and c['component_length'] <= limit_30 , unitigs))
    large = list(filter(lambda c: c['component_length'] > limit_30, unitigs))
    
    small_rel_improvements_base = list(map(lambda c: c['rel_improvements_base'] , small))
    medium_rel_improvements_base = list(map(lambda c: c['rel_improvements_base'] , medium))
    large_rel_improvements_base = list(map(lambda c: c['rel_improvements_base'] , large))
    total_rel_improvements_base = list(map(lambda c: c['rel_improvements_base'] , unitigs))
    
    small_rel_improvements_vertex = list(map(lambda c: c['rel_improvements_vertex'] , small))
    medium_rel_improvements_vertex = list(map(lambda c: c['rel_improvements_vertex'] , medium))
    large_rel_improvements_vertex = list(map(lambda c: c['rel_improvements_vertex'] , large))
    total_rel_improvements_vertex = list(map(lambda c: c['rel_improvements_vertex'] , unitigs))
    
    
    
    
    return { 
        'small':
        {
            'rel_improvements_vertex': 
            {
                'median': median(small_rel_improvements_vertex),
                'mean': mean(small_rel_improvements_vertex),
                'stdev': stdev(small_rel_improvements_vertex)
            },
            'rel_improvements_base':
            {
                'median': median(small_rel_improvements_base),
                'mean': mean(small_rel_improvements_base),
                'stdev': stdev(small_rel_improvements_base)
            },
            'number_of_unitigs': len(small)            
        },
        'medium':
        {
            'rel_improvements_vertex': 
            {
                'median': median(medium_rel_improvements_vertex),
                'mean': mean(medium_rel_improvements_vertex),
                'stdev': stdev(medium_rel_improvements_vertex)
            },
            'rel_improvements_base':
            {
                'median': median(medium_rel_improvements_base),
                'mean': mean(medium_rel_improvements_base),
                'stdev': stdev(medium_rel_improvements_base)
            },
            'number_of_unitigs': len(medium)
            
        },
        'large':
        {
            'rel_improvements_vertex': 
            {
                'median': median(large_rel_improvements_vertex),
                'mean': mean(large_rel_improvements_vertex),
                'stdev': stdev(large_rel_improvements_vertex)
            },
            'rel_improvements_base':
            {
                'median': median(large_rel_improvements_base),
                'mean': mean(large_rel_improvements_base),
                'stdev': stdev(large_rel_improvements_base)
            },
            'number_of_unitigs': len(large)
            
        },
        'total':
        {
            'rel_improvements_vertex': 
            {
                'median': median(total_rel_improvements_vertex),
                'mean': mean(total_rel_improvements_vertex),
                'stdev': stdev(total_rel_improvements_vertex)
            },
            'rel_improvements_base':
            {
                'median': median(total_rel_improvements_base),
                'mean': mean(total_rel_improvements_base),
                'stdev': stdev(total_rel_improvements_base)
            },
            'number_of_unitigs': len(unitigs)
            
        }
    }
    
pprint(compute_rel_improvement_table(-1, limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'number_of_unitigs': 23857,
           'rel_improvements_base': {'mean': 1.9364123011041254,
                                     'median': 1.1093117408906883,
                                     'stdev': 3.796463199470008},
           'rel_improvements_vertex': {'mean': 1.523322717646293,
                                       'median': 1.2857142857142858,
                                       'stdev': 0.9170090619844139}},
 'medium': {'number_of_unitigs': 32246,
            'rel_improvements_base': {'mean': 1.817037117189901,
                                      'median': 1.056030188605891,
                                      'stdev': 4.5078974380953705},
            'rel_improvements_vertex': {'mean': 1.4544017553372026,
                                        'median': 1.2,
                                        'stdev': 0.7585772731196603}},
 'small': {'number_of_unitigs': 18852,
           'rel_improvements_base': {'mean': 1.4235048754418045,
                    

In [50]:
def compute_contigs_table(over_width, limit_60, limit_30, components, vertices_inv):
    contigs = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)
                
            if component['experiments'].get(l, None) is not None:
                safe_paths_info = component['experiments'][l]
                
                for safe_path in safe_paths_info['safe_paths']:
                    d = dict()
                    d['component'] = i
                    d['component_length'] = component['len']
                    d['vertices'] = len(safe_path)
                    d['length'] = base_length(safe_path, vertices_inv)
                    
                    contigs.append(d)

                
                    
    
    
   
    small = list(filter(lambda c: c['component_length'] <= limit_60, contigs))
    medium = list(filter(lambda c: c['component_length'] > limit_60 and c['component_length'] <= limit_30 , contigs))
    large = list(filter(lambda c: c['component_length'] > limit_30, contigs))
    
    small_base_lengths = list(map(lambda c: c['length'] , small))
    medium_base_lengths = list(map(lambda c: c['length'] , medium))
    large_base_lengths = list(map(lambda c: c['length'] , large))
    total_base_lengths = list(map(lambda c: c['length'] , contigs))
    
    small_vertex_lengths = list(map(lambda c: c['vertices'] , small))
    medium_vertex_lengths = list(map(lambda c: c['vertices'] , medium))
    large_vertex_lengths = list(map(lambda c: c['vertices'] , large))
    total_vertex_lengths = list(map(lambda c: c['vertices'] , contigs))
    
    
    
    
    return { 
        'small':
        {
            'vertex_lengths': 
            {
                'median': median(small_vertex_lengths),
                'mean': mean(small_vertex_lengths),
                'stdev': stdev(small_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(small_base_lengths),
                'mean': mean(small_base_lengths),
                'stdev': stdev(small_base_lengths)
            },
            'number_of_contigs': len(small)            
        },
        'medium':
        {
            'vertex_lengths': 
            {
                'median': median(medium_vertex_lengths),
                'mean': mean(medium_vertex_lengths),
                'stdev': stdev(medium_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(medium_base_lengths),
                'mean': mean(medium_base_lengths),
                'stdev': stdev(medium_base_lengths)
            },
            'number_of_contigs': len(medium)
            
        },
        'large':
        {
            'vertex_lengths': 
            {
                'median': median(large_vertex_lengths),
                'mean': mean(large_vertex_lengths),
                'stdev': stdev(large_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(large_base_lengths),
                'mean': mean(large_base_lengths),
                'stdev': stdev(large_base_lengths)
            },
            'number_of_contigs': len(large)
            
        },
        'total':
        {
            'vertex_lengths': 
            {
                'median': median(total_vertex_lengths),
                'mean': mean(total_vertex_lengths),
                'stdev': stdev(total_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(total_base_lengths),
                'mean': mean(total_base_lengths),
                'stdev': stdev(total_base_lengths)
            },
            'number_of_contigs': len(contigs)
            
        }
    }
    
pprint(compute_contigs_table(-1, limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'base_lengths': {'mean': 671.8732810397453,
                            'median': 402,
                            'stdev': 938.2866812845797},
           'number_of_contigs': 30469,
           'vertex_lengths': {'mean': 4.696248646164954,
                              'median': 4,
                              'stdev': 3.5750445206749824}},
 'medium': {'base_lengths': {'mean': 767.3544157800584,
                             'median': 449.0,
                             'stdev': 1057.421043861511},
            'number_of_contigs': 42484,
            'vertex_lengths': {'mean': 4.45304114490161,
                               'median': 4.0,
                               'stdev': 2.7708390549912156}},
 'small': {'base_lengths': {'mean': 1018.3653104925054,
                            'median': 583.0,
                            'stdev': 1484.0827219399116},
           'number_of_contigs': 28020,
           'vertex_lengths': {'mean': 3.69768022840828,
                          

In [55]:
def compute_contigs_table_unitigs(limit_60, limit_30, components, vertices_inv):
    contigs = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            unitigs = component['unitigs']

            for unitig in unitigs:
                d = dict()
                d['component'] = i
                d['component_length'] = component['len']
                d['vertices'] = len(unitig)
                d['length'] = base_length(unitig, vertices_inv)

                contigs.append(d)
                
    
    
    small = list(filter(lambda c: c['component_length'] <= limit_60, contigs))
    medium = list(filter(lambda c: c['component_length'] > limit_60 and c['component_length'] <= limit_30 , contigs))
    large = list(filter(lambda c: c['component_length'] > limit_30, contigs))
    
    small_base_lengths = list(map(lambda c: c['length'] , small))
    medium_base_lengths = list(map(lambda c: c['length'] , medium))
    large_base_lengths = list(map(lambda c: c['length'] , large))
    total_base_lengths = list(map(lambda c: c['length'] , contigs))
    
    small_vertex_lengths = list(map(lambda c: c['vertices'] , small))
    medium_vertex_lengths = list(map(lambda c: c['vertices'] , medium))
    large_vertex_lengths = list(map(lambda c: c['vertices'] , large))
    total_vertex_lengths = list(map(lambda c: c['vertices'] , contigs))
    
    
    
    
    return { 
        'small':
        {
            'vertex_lengths': 
            {
                'median': median(small_vertex_lengths),
                'mean': mean(small_vertex_lengths),
                'stdev': stdev(small_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(small_base_lengths),
                'mean': mean(small_base_lengths),
                'stdev': stdev(small_base_lengths)
            },
            'number_of_contigs': len(small)            
        },
        'medium':
        {
            'vertex_lengths': 
            {
                'median': median(medium_vertex_lengths),
                'mean': mean(medium_vertex_lengths),
                'stdev': stdev(medium_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(medium_base_lengths),
                'mean': mean(medium_base_lengths),
                'stdev': stdev(medium_base_lengths)
            },
            'number_of_contigs': len(medium)
            
        },
        'large':
        {
            'vertex_lengths': 
            {
                'median': median(large_vertex_lengths),
                'mean': mean(large_vertex_lengths),
                'stdev': stdev(large_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(large_base_lengths),
                'mean': mean(large_base_lengths),
                'stdev': stdev(large_base_lengths)
            },
            'number_of_contigs': len(large)
            
        },
        'total':
        {
            'vertex_lengths': 
            {
                'median': median(total_vertex_lengths),
                'mean': mean(total_vertex_lengths),
                'stdev': stdev(total_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(total_base_lengths),
                'mean': mean(total_base_lengths),
                'stdev': stdev(total_base_lengths)
            },
            'number_of_contigs': len(contigs)
            
        }
    }

pprint(compute_contigs_table_unitigs(limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'base_lengths': {'mean': 578.9776585488536,
                            'median': 318,
                            'stdev': 927.4126735518306},
           'number_of_contigs': 23857,
           'vertex_lengths': {'mean': 4.342415224043258,
                              'median': 3,
                              'stdev': 3.1077091058801223}},
 'medium': {'base_lengths': {'mean': 696.6927370836693,
                             'median': 363.0,
                             'stdev': 1068.2320941184112},
            'number_of_contigs': 32246,
            'vertex_lengths': {'mean': 4.148297463251256,
                               'median': 3.0,
                               'stdev': 2.2844871929068025}},
 'small': {'base_lengths': {'mean': 1004.8013473371526,
                            'median': 549.5,
                            'stdev': 1317.6242438288753},
           'number_of_contigs': 18852,
           'vertex_lengths': {'mean': 3.752227880330999,
                       

In [54]:
def compute_fixed_l_table(over_width, limit_60, limit_30, components, vertices_inv):
    transcripts = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)
                
            if component['experiments'].get(l, None) is not None:
                safe_paths_info = component['experiments'][l]

                for j, transcript in enumerate(component['transcript_paths']):
                    d = dict()
                    d['component'] = i
                    d['length'] = base_length(transcript, vertices_inv)
                    d['transcript'] = transcript
                    
                    d['e_size_density_vertex'] = safe_paths_info['e_sizes_vertex'][j]/len(transcript)
                    d['e_size_density_bases'] = safe_paths_info['e_size_bases'][j]/d['length']
                    d['max_prop_cov_bases'] = safe_paths_info['max_cov_bases'][j]/d['length']
                    d['max_prop_cov_vertex'] = safe_paths_info['max_cov_vertex'][j]/len(transcript)
                    
                    transcripts.append(d)
    
    
    
    small = list(filter(lambda t: t['length'] <= limit_60, transcripts))
    medium = list(filter(lambda t: t['length'] > limit_60 and t['length'] <= limit_30 , transcripts))
    large = list(filter(lambda t: t['length'] > limit_30, transcripts))
    
    
    small_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , small))
    medium_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , medium))
    large_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , large))
    
    small_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , small))
    medium_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , medium))
    large_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , large))
    
    
    small_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , small))
    medium_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , medium))
    large_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , large))
    
    small_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , small))
    medium_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , medium))
    large_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , large))
    
    
    
    return { 
        'small':
        {
            'e_size_density_vertex': 
            {
                'median': median(small_e_size_densities_vertex),
                'mean': mean(small_e_size_densities_vertex),
                'stdev': stdev(small_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(small_e_size_densities_bases),
                'mean': mean(small_e_size_densities_bases),
                'stdev': stdev(small_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(small_max_prop_cov_vertex),
                'mean': mean(small_max_prop_cov_vertex),
                'stdev': stdev(small_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(small_max_prop_cov_bases),
                'mean': mean(small_max_prop_cov_bases),
                'stdev': stdev(small_max_prop_cov_bases)
            }
            
        }
        ,
        'medium' :
        {
            'e_size_density_vertex': 
            {
                'median': median(medium_e_size_densities_vertex),
                'mean': mean(medium_e_size_densities_vertex),
                'stdev': stdev(medium_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(medium_e_size_densities_bases),
                'mean': mean(medium_e_size_densities_bases),
                'stdev': stdev(medium_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(medium_max_prop_cov_vertex),
                'mean': mean(medium_max_prop_cov_vertex),
                'stdev': stdev(medium_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(medium_max_prop_cov_bases),
                'mean': mean(medium_max_prop_cov_bases),
                'stdev': stdev(medium_max_prop_cov_bases)
            }
        },
        'large':
        {
            'e_size_density_vertex': 
            {
                'median': median(large_e_size_densities_vertex),
                'mean': mean(large_e_size_densities_vertex),
                'stdev': stdev(large_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(large_e_size_densities_bases),
                'mean': mean(large_e_size_densities_bases),
                'stdev': stdev(large_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(large_max_prop_cov_vertex),
                'mean': mean(large_max_prop_cov_vertex),
                'stdev': stdev(large_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(large_max_prop_cov_bases),
                'mean': mean(large_max_prop_cov_bases),
                'stdev': stdev(large_max_prop_cov_bases)
            }
        }
    }
    
pprint(compute_fixed_l_table(0, limit_60, limit_30, components, vertices_inv))

{'large': {'e_size_density_bases': {'mean': 0.48843401085056903,
                                    'median': 0.4490438294731364,
                                    'stdev': 0.2540928807513626},
           'e_size_density_vertex': {'mean': 0.3498919297198766,
                                     'median': 0.28656462585034015,
                                     'stdev': 0.22075548509770684},
           'max_prop_cov_bases': {'mean': 0.6253037065305191,
                                  'median': 0.618051013734467,
                                  'stdev': 0.23441831569455798},
           'max_prop_cov_vertex': {'mean': 0.4794598252949863,
                                   'median': 0.43333333333333335,
                                   'stdev': 0.22080904137393437}},
 'medium': {'e_size_density_bases': {'mean': 0.47818762396115516,
                                     'median': 0.43225340871739565,
                                     'stdev': 0.24583830816911673},
            'e

In [57]:
def compute_fixed_l_table_unitigs(limit_60, limit_30, components, vertices_inv):
    transcripts = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            unitigs = component['unitigs']

            for j, transcript in enumerate(component['transcript_paths']):
                d = dict()
                d['component'] = i
                d['length'] = base_length(transcript, vertices_inv)
                d['transcript'] = transcript
                
                
                d['e_size_density_vertex'] =  component['e_sizes_vertex'][j]/len(transcript)
                d['e_size_density_bases'] =  component['e_size_bases'][j]/d['length']
                d['max_prop_cov_bases'] =  component['max_cov_bases'][j]/d['length']
                d['max_prop_cov_vertex'] =  component['max_cov_vertex'][j]/len(transcript)

                transcripts.append(d)

    
    
    small = list(filter(lambda t: t['length'] <= limit_60, transcripts))
    medium = list(filter(lambda t: t['length'] > limit_60 and t['length'] <= limit_30 , transcripts))
    large = list(filter(lambda t: t['length'] > limit_30, transcripts))
    
    
    small_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , small))
    medium_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , medium))
    large_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , large))
    
    small_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , small))
    medium_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , medium))
    large_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , large))
    
    
    small_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , small))
    medium_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , medium))
    large_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , large))
    
    small_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , small))
    medium_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , medium))
    large_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , large))

    
    
    return { 
        'small':
        {
            'e_size_density_vertex': 
            {
                'median': median(small_e_size_densities_vertex),
                'mean': mean(small_e_size_densities_vertex),
                'stdev': stdev(small_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(small_e_size_densities_bases),
                'mean': mean(small_e_size_densities_bases),
                'stdev': stdev(small_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(small_max_prop_cov_vertex),
                'mean': mean(small_max_prop_cov_vertex),
                'stdev': stdev(small_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(small_max_prop_cov_bases),
                'mean': mean(small_max_prop_cov_bases),
                'stdev': stdev(small_max_prop_cov_bases)
            }
            
        }
        ,
        'medium' :
        {
            'e_size_density_vertex': 
            {
                'median': median(medium_e_size_densities_vertex),
                'mean': mean(medium_e_size_densities_vertex),
                'stdev': stdev(medium_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(medium_e_size_densities_bases),
                'mean': mean(medium_e_size_densities_bases),
                'stdev': stdev(medium_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(medium_max_prop_cov_vertex),
                'mean': mean(medium_max_prop_cov_vertex),
                'stdev': stdev(medium_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(medium_max_prop_cov_bases),
                'mean': mean(medium_max_prop_cov_bases),
                'stdev': stdev(medium_max_prop_cov_bases)
            }
        },
        'large':
        {
            'e_size_density_vertex': 
            {
                'median': median(large_e_size_densities_vertex),
                'mean': mean(large_e_size_densities_vertex),
                'stdev': stdev(large_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(large_e_size_densities_bases),
                'mean': mean(large_e_size_densities_bases),
                'stdev': stdev(large_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(large_max_prop_cov_vertex),
                'mean': mean(large_max_prop_cov_vertex),
                'stdev': stdev(large_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(large_max_prop_cov_bases),
                'mean': mean(large_max_prop_cov_bases),
                'stdev': stdev(large_max_prop_cov_bases)
            }
        }
    }

pprint(compute_fixed_l_table_unitigs(limit_60, limit_30, components, vertices_inv))

{'large': {'e_size_density_bases': {'mean': 0.3806891567877237,
                                    'median': 0.3207514617051683,
                                    'stdev': 0.2530305596698371},
           'e_size_density_vertex': {'mean': 0.24943402182486896,
                                     'median': 0.19576333089846604,
                                     'stdev': 0.18057871669076303},
           'max_prop_cov_bases': {'mean': 0.5254454879086077,
                                  'median': 0.5024841915085817,
                                  'stdev': 0.24772857607787951},
           'max_prop_cov_vertex': {'mean': 0.36799846511734646,
                                   'median': 0.32,
                                   'stdev': 0.19494554032134626}},
 'medium': {'e_size_density_bases': {'mean': 0.3490242925939059,
                                     'median': 0.2864200598143168,
                                     'stdev': 0.24326912322761918},
            'e_size_density_v

In [59]:
def compute_fixed_rd_table(over_width, limit_60, limit_30, components, vertices_inv):
    genes = list()
    transcripts = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
                
                
            if component['experiments'].get(l, None) is None:
                l = str(2*width)
                        
            if component['experiments'].get(l, None) is not None:
                safe_paths_info = component['experiments'][l]

                for j, transcript in enumerate(component['transcript_paths']):
                    d = dict()
                    d['component'] = i
                    d['component_length'] = component['len']
                    d['length'] = base_length(transcript, vertices_inv)
                    d['transcript'] = transcript
                    
                    
                    d['e_size_density_vertex'] = safe_paths_info['e_sizes_vertex'][j]/len(transcript)
                    d['e_size_density_bases'] = safe_paths_info['e_size_bases'][j]/d['length']
                    d['max_prop_cov_bases'] = safe_paths_info['max_cov_bases'][j]/d['length']
                    d['max_prop_cov_vertex'] = safe_paths_info['max_cov_vertex'][j]/len(transcript)
                    
                    transcripts.append(d)
                 
                d = dict()
                
                tp = 0
                for safe_path in safe_paths_info['true_positives']:
                    tp += base_length(safe_path, vertices_inv)
                    
                p = tp
                for safe_path in safe_paths_info['false_positives']:
                    p += base_length(safe_path, vertices_inv)
                
                d['precision'] = tp/p
                d['component_length'] = components[i]['len']
                genes.append(d)
    
    small = list(filter(lambda t: t['component_length'] <= limit_60, transcripts))
    medium = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , transcripts))
    large = list(filter(lambda t: t['component_length'] > limit_30, transcripts))
    
    small_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , small))
    medium_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , medium))
    large_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , large))
    
    small_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , small))
    medium_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , medium))
    large_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , large))
    
    
    small_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , small))
    medium_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , medium))
    large_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , large))
    
    small_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , small))
    medium_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , medium))
    large_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , large))
    
    
    s = list(filter(lambda t: t['component_length'] <= limit_60, genes))
    m = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , genes))
    l = list(filter(lambda t: t['component_length'] > limit_30, genes))
    
    s_precisions = list(map(lambda t: t['precision'] , s))
    m_precisions = list(map(lambda t: t['precision'] , m))
    l_precisions = list(map(lambda t: t['precision'] , l))
    
    
    return { 
        'small':
        {
            'e_size_density_vertex': 
            {
                'median': median(small_e_size_densities_vertex),
                'mean': mean(small_e_size_densities_vertex),
                'stdev': stdev(small_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(small_e_size_densities_bases),
                'mean': mean(small_e_size_densities_bases),
                'stdev': stdev(small_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(small_max_prop_cov_vertex),
                'mean': mean(small_max_prop_cov_vertex),
                'stdev': stdev(small_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(small_max_prop_cov_bases),
                'mean': mean(small_max_prop_cov_bases),
                'stdev': stdev(small_max_prop_cov_bases)
            },
            'precision':
            {
                'median': median(s_precisions),
                'mean': mean(s_precisions),
                'stdev': stdev(s_precisions)
            }
            
        }
        ,
        'medium' :
        {
            'e_size_density_vertex': 
            {
                'median': median(medium_e_size_densities_vertex),
                'mean': mean(medium_e_size_densities_vertex),
                'stdev': stdev(medium_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(medium_e_size_densities_bases),
                'mean': mean(medium_e_size_densities_bases),
                'stdev': stdev(medium_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(medium_max_prop_cov_vertex),
                'mean': mean(medium_max_prop_cov_vertex),
                'stdev': stdev(medium_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(medium_max_prop_cov_bases),
                'mean': mean(medium_max_prop_cov_bases),
                'stdev': stdev(medium_max_prop_cov_bases)
            },
            'precision':
            {
                'median': median(m_precisions),
                'mean': mean(m_precisions),
                'stdev': stdev(m_precisions)
            }
        },
        'large':
        {
            'e_size_density_vertex': 
            {
                'median': median(large_e_size_densities_vertex),
                'mean': mean(large_e_size_densities_vertex),
                'stdev': stdev(large_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(large_e_size_densities_bases),
                'mean': mean(large_e_size_densities_bases),
                'stdev': stdev(large_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(large_max_prop_cov_vertex),
                'mean': mean(large_max_prop_cov_vertex),
                'stdev': stdev(large_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(large_max_prop_cov_bases),
                'mean': mean(large_max_prop_cov_bases),
                'stdev': stdev(large_max_prop_cov_bases)
            },
            'precision':
            {
                'median': median(l_precisions),
                'mean': mean(l_precisions),
                'stdev': stdev(l_precisions)
            }
        }
    }
    
pprint(compute_fixed_rd_table(0, limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'e_size_density_bases': {'mean': 0.41714149761872815,
                                    'median': 0.36897707834567395,
                                    'stdev': 0.21855757729032965},
           'e_size_density_vertex': {'mean': 0.3728023373605343,
                                     'median': 0.32653061224489793,
                                     'stdev': 0.2046219563447882},
           'max_prop_cov_bases': {'mean': 0.5629802809411107,
                                  'median': 0.532770637138839,
                                  'stdev': 0.2231472345191591},
           'max_prop_cov_vertex': {'mean': 0.508781466766869,
                                   'median': 0.47619047619047616,
                                   'stdev': 0.21553322358998842},
           'precision': {'mean': 0.847837431112713,
                         'median': 0.8995228234446745,
                         'stdev': 0.16750627150093056}},
 'medium': {'e_size_density_bases': {'mean': 0.5005563

In [60]:
def compute_fixed_rd_table_unitigs(limit_60, limit_30, components, vertices_inv):
    transcripts = list()
    genes = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            unitigs = component['unitigs']

            for j, transcript in enumerate(component['transcript_paths']):
                d = dict()
                d['component'] = i
                d['component_length'] = components[i]['len']
                d['length'] = base_length(transcript, vertices_inv)
                d['transcript'] = transcript
                
                
                d['e_size_density_vertex'] =  component['e_sizes_vertex'][j]/len(transcript)
                d['e_size_density_bases'] =  component['e_size_bases'][j]/d['length']
                d['max_prop_cov_bases'] =  component['max_cov_bases'][j]/d['length']
                d['max_prop_cov_vertex'] =  component['max_cov_vertex'][j]/len(transcript)

                transcripts.append(d)
                
            d = dict()
                
            tp = 0
            for safe_path in component['true_positives']:
                tp += base_length(safe_path, vertices_inv)

            p = tp
            for safe_path in component['false_positives']:
                p += base_length(safe_path, vertices_inv)
                

            d['precision'] = 1 if p==0 else tp/p
            d['component_length'] = component['len']
            genes.append(d)

    
    
    small = list(filter(lambda t: t['component_length'] <= limit_60, transcripts))
    medium = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , transcripts))
    large = list(filter(lambda t: t['component_length'] > limit_30, transcripts))
    
    small_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , small))
    medium_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , medium))
    large_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , large))
    
    small_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , small))
    medium_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , medium))
    large_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , large))
    
    
    small_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , small))
    medium_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , medium))
    large_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , large))
    
    small_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , small))
    medium_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , medium))
    large_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , large))
    
    
    s = list(filter(lambda t: t['component_length'] <= limit_60, genes))
    m = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , genes))
    l = list(filter(lambda t: t['component_length'] > limit_30, genes))
    
    s_precisions = list(map(lambda t: t['precision'] , s))
    m_precisions = list(map(lambda t: t['precision'] , m))
    l_precisions = list(map(lambda t: t['precision'] , l))
    
    
    return { 
        'small':
        {
            'e_size_density_vertex': 
            {
                'median': median(small_e_size_densities_vertex),
                'mean': mean(small_e_size_densities_vertex),
                'stdev': stdev(small_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(small_e_size_densities_bases),
                'mean': mean(small_e_size_densities_bases),
                'stdev': stdev(small_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(small_max_prop_cov_vertex),
                'mean': mean(small_max_prop_cov_vertex),
                'stdev': stdev(small_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(small_max_prop_cov_bases),
                'mean': mean(small_max_prop_cov_bases),
                'stdev': stdev(small_max_prop_cov_bases)
            },
            'precision':
            {
                'median': median(s_precisions),
                'mean': mean(s_precisions),
                'stdev': stdev(s_precisions)
            }
            
        }
        ,
        'medium' :
        {
            'e_size_density_vertex': 
            {
                'median': median(medium_e_size_densities_vertex),
                'mean': mean(medium_e_size_densities_vertex),
                'stdev': stdev(medium_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(medium_e_size_densities_bases),
                'mean': mean(medium_e_size_densities_bases),
                'stdev': stdev(medium_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(medium_max_prop_cov_vertex),
                'mean': mean(medium_max_prop_cov_vertex),
                'stdev': stdev(medium_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(medium_max_prop_cov_bases),
                'mean': mean(medium_max_prop_cov_bases),
                'stdev': stdev(medium_max_prop_cov_bases)
            },
            'precision':
            {
                'median': median(m_precisions),
                'mean': mean(m_precisions),
                'stdev': stdev(m_precisions)
            }
        },
        'large':
        {
            'e_size_density_vertex': 
            {
                'median': median(large_e_size_densities_vertex),
                'mean': mean(large_e_size_densities_vertex),
                'stdev': stdev(large_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(large_e_size_densities_bases),
                'mean': mean(large_e_size_densities_bases),
                'stdev': stdev(large_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(large_max_prop_cov_vertex),
                'mean': mean(large_max_prop_cov_vertex),
                'stdev': stdev(large_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(large_max_prop_cov_bases),
                'mean': mean(large_max_prop_cov_bases),
                'stdev': stdev(large_max_prop_cov_bases)
            },
            'precision':
            {
                'median': median(l_precisions),
                'mean': mean(l_precisions),
                'stdev': stdev(l_precisions)
            }
        }
    }

pprint(compute_fixed_rd_table_unitigs(limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'e_size_density_bases': {'mean': 0.2767474744422535,
                                    'median': 0.23109977910954016,
                                    'stdev': 0.18777367625935018},
           'e_size_density_vertex': {'mean': 0.2614866277899087,
                                     'median': 0.22453703703703706,
                                     'stdev': 0.15769141207401716},
           'max_prop_cov_bases': {'mean': 0.42886139787405575,
                                  'median': 0.39526625377361424,
                                  'stdev': 0.20256202030795475},
           'max_prop_cov_vertex': {'mean': 0.4025184280266759,
                                   'median': 0.375,
                                   'stdev': 0.1853223535658438},
           'precision': {'mean': 1.0, 'median': 1.0, 'stdev': 0.0}},
 'medium': {'e_size_density_bases': {'mean': 0.33815263847544497,
                                     'median': 0.30365279022051384,
                         

In [79]:
def compute_time_table(over_width, limit_60, limit_30, components, vertices_inv, algorithm_experiments='experiments'):
    time = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component[algorithm_experiments].get(l, None) is not None:
                safe_paths_info = component[algorithm_experiments][l]

                d = dict()
                d['time'] = component[algorithm_experiments][l]['time_main']+component[algorithm_experiments][l]['time_filter']
                print(d['time'])
                break
                d['component_length'] = component['len']
                time.append(d)
    
    small = list(filter(lambda t: t['component_length'] <= limit_60, time))
    medium = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , time))
    large = list(filter(lambda t: t['component_length'] > limit_30, time))
    
    return { 
        'small':
        {
            'time': sum(list(map(lambda x: x['time'] , small)))
        }
        ,
        'medium' :
        {
            'time': sum(list(map(lambda x: x['time'] , medium)))
        },
        'large':
        {
            'time': sum(list(map(lambda x: x['time'] , large)))
        },
        'all':
        {
            'time': sum(list(map(lambda x: x['time'] , time)))
        }
    }

pprint(compute_time_table(0, limit_size_60, limit_size_30, components, vertices_inv))
pprint(compute_time_table(0, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_naive'))

161
{'all': {'time': 0},
 'large': {'time': 0},
 'medium': {'time': 0},
 'small': {'time': 0}}
{'all': {'time': 0},
 'large': {'time': 0},
 'medium': {'time': 0},
 'small': {'time': 0}}


In [78]:
## And peak memory
max(list(map(lambda x: 0 if x.get('experiments', None) is None else max(list(map(lambda k: x['experiments'][k]['peak_memory'], x['experiments'].keys()))),
             components))), max(list(map(lambda x: 0 if x.get('experiments_naive', None) is None else max(list(map(lambda k: x['experiments_naive'][k]['peak_mem'], x['experiments_naive'].keys()))),
             components)))

(5260, 4892)